In [ ]:
"""
K-MEANS CLUSTERING: needs improvemnt
problems include:
1. Only 2 clusters (K=2) - too simplistic
2. No connection to recession periods
5. Weak silhouette scores (0.35)

SOLUTIONS:
"""


import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score, calinski_harabasz_score
import matplotlib.pyplot as plt
import seaborn as sns

# Load data
df = pd.read_csv('music_econ_topics_merged.csv')

feature_cols = ['danceability', 'energy', 'valence', 'tempo', 
                'acousticness', 'instrumentalness', 'speechiness', 'loudness']

df_clean = df[feature_cols + ['week_date', 'USREC']].dropna()

# Standardize
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_clean[feature_cols])


# Test multiple metrics
inertias = []
silhouette_scores = []
calinski_scores = []
K_range = range(2, 11)

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = kmeans.fit_predict(X_scaled)
    
    inertias.append(kmeans.inertia_)
    silhouette_scores.append(silhouette_score(X_scaled, labels))
    calinski_scores.append(calinski_harabasz_score(X_scaled, labels))
    
    print(f"K={k}: Silhouette={silhouette_scores[-1]:.3f}, Calinski-Harabasz={calinski_scores[-1]:.1f}")

# USE K=3 OR K=4 (MORE INTERPRETABLE)

OPTIMAL_K = 3  

kmeans_final = KMeans(n_clusters=OPTIMAL_K, random_state=42, n_init=20)
df_clean['cluster'] = kmeans_final.fit_predict(X_scaled)

print(f"\n✓ Final model: K={OPTIMAL_K}")
print(f"✓ Silhouette score: {silhouette_score(X_scaled, df_clean['cluster']):.3f}")

#CREATE MEANINGFUL CLUSTER LABELS

cluster_profiles = df_clean.groupby('cluster')[feature_cols].mean()

def create_meaningful_labels(cluster_profiles):
    """
    Create interpretable labels based on MULTIPLE features
    """
    labels = {}
    
    for cluster in range(OPTIMAL_K):
        profile = cluster_profiles.loc[cluster]
        mean = cluster_profiles.mean()
        
        # Identify key characteristics (z-score > 0.5)
        characteristics = []
        
        # Energy/Tempo
        if profile['energy'] > mean['energy'] + 0.1 and profile['tempo'] > mean['tempo'] + 5:
            characteristics.append("High-Energy/Fast")
        elif profile['energy'] < mean['energy'] - 0.1:
            characteristics.append("Low-Energy")
            
        # Acousticness
        if profile['acousticness'] > mean['acousticness'] + 0.1:
            characteristics.append("Acoustic")
        elif profile['acousticness'] < mean['acousticness'] - 0.05:
            characteristics.append("Electric")
            
        # Danceability
        if profile['danceability'] > mean['danceability'] + 0.05:
            characteristics.append("Danceable")
        elif profile['danceability'] < mean['danceability'] - 0.05:
            characteristics.append("Non-Danceable")
            
        # Valence
        if profile['valence'] > mean['valence'] + 0.05:
            characteristics.append("Positive")
        elif profile['valence'] < mean['valence'] - 0.05:
            characteristics.append("Melancholic")
        
        # Create label
        if len(characteristics) == 0:
            labels[cluster] = "Balanced/Moderate"
        else:
            labels[cluster] = " / ".join(characteristics[:2])  # Top 2 characteristics
    
    return labels

cluster_labels = create_meaningful_labels(cluster_profiles)



for cluster in range(OPTIMAL_K):
    count = (df_clean['cluster'] == cluster).sum()
    pct = count / len(df_clean) * 100
    print(f"Cluster {cluster}: {cluster_labels[cluster]:<30} ({count} months, {pct:.1f}%)")

# found these to be best labels
cluster_labels = {
    0: "Balanced/Mid-Energy",
    1: "High-Energy/Dance", 
    2: "Acoustic/Positive"
}





for cluster in range(OPTIMAL_K):
    recession_mask = (df_clean['cluster'] == cluster) & (df_clean['USREC'] == 1)
    normal_mask = (df_clean['cluster'] == cluster) & (df_clean['USREC'] == 0)
    
    recession_count = recession_mask.sum()
    normal_count = normal_mask.sum()
    
    total_recessions = (df_clean['USREC'] == 1).sum()
    total_normal = (df_clean['USREC'] == 0).sum()
    
    recession_pct = (recession_count / total_recessions) * 100 if total_recessions > 0 else 0
    normal_pct = (normal_count / total_normal) * 100 if total_normal > 0 else 0
    
    diff = recession_pct - normal_pct
    
    print(f"\nCluster {cluster}: {cluster_labels[cluster]}")
    print(f"  Recession periods: {recession_count:2d} ({recession_pct:5.1f}% of all recession months)")
    print(f"  Normal periods:    {normal_count:3d} ({normal_pct:5.1f}% of all normal months)")
    print(f"  Difference:        {diff:+5.1f} percentage points", end="")
    
    if abs(diff) > 5:
        if diff > 0:
            print(" ⬆️ MORE COMMON IN RECESSIONS")
        else:
            print(" ⬇️ MORE COMMON IN NORMAL TIMES")
    else:
        print(" → SIMILAR")

# STATISTICAL TEST

from scipy.stats import chi2_contingency

# make contingency table
contingency_table = pd.crosstab(df_clean['cluster'], df_clean['USREC'])
chi2, p_value, dof, expected = chi2_contingency(contingency_table)


print(f"Chi-square statistic: χ² = {chi2:.4f}")
print(f"P-value: p = {p_value:.6f}")
print(f"Degrees of freedom: df = {dof}")

if p_value < 0.05:
    print("✅ SIGNIFICANT: Cluster distribution differs between recession/normal periods")
else:
    print("❌ NOT SIGNIFICANT: No strong evidence clusters relate to recessions")

# visuals for poster

# 1. Heatmap of cluster characteristics
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
ax = axes[0, 0]
sns.heatmap(cluster_profiles.T, annot=True, fmt='.3f', cmap='RdYlGn',
            ax=ax, cbar_kws={'label': 'Feature Value'})
ax.set_title('Cluster Profiles: Audio Features', fontsize=14, fontweight='bold')
ax.set_xlabel('Cluster')
ax.set_ylabel('Audio Feature')
ax.set_xticklabels([cluster_labels[i] for i in range(OPTIMAL_K)], rotation=45, ha='right')

ax = axes[0, 1]

recession_dist = []
normal_dist = []

for cluster in range(OPTIMAL_K):
    # Count how many months in this cluster are recession vs normal
    recession_count = ((df_clean['cluster'] == cluster) & (df_clean['USREC'] == 1)).sum()
    normal_count = ((df_clean['cluster'] == cluster) & (df_clean['USREC'] == 0)).sum()
    
    recession_dist.append(recession_count)
    normal_dist.append(normal_count)

x = np.arange(OPTIMAL_K)
width = 0.35

ax.bar(x - width/2, recession_dist, width, label='Recession', color='darkred', alpha=0.8)
ax.bar(x + width/2, normal_dist, width, label='Normal', color='darkblue', alpha=0.8)

ax.set_xlabel('Cluster', fontsize=12, fontweight='bold')
ax.set_ylabel('Number of Months', fontsize=12, fontweight='bold')
ax.set_title('Cluster Distribution: Recession vs Normal', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels([cluster_labels[i] for i in range(OPTIMAL_K)], rotation=45, ha='right')
ax.legend()
ax.grid(True, alpha=0.3, axis='y')

# Bottom-left: PCA visualization
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

ax = axes[1, 0]
colors = ['red', 'blue', 'green', 'purple', 'orange']
for cluster in range(OPTIMAL_K):
    mask = df_clean['cluster'] == cluster
    ax.scatter(X_pca[mask, 0], X_pca[mask, 1], 
              c=colors[cluster], label=cluster_labels[cluster], alpha=0.6, s=50)
ax.set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]*100:.1f}%)')
ax.set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]*100:.1f}%)')
ax.set_title('K-means Clusters (PCA Projection)', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

# Bottom-right: Feature importance (distance from center)
ax = axes[1, 1]
feature_importance = np.abs(cluster_profiles - cluster_profiles.mean()).mean()
feature_importance = feature_importance.sort_values(ascending=True)
ax.barh(feature_importance.index, feature_importance.values, color='teal', alpha=0.7)
ax.set_xlabel('Mean Absolute Deviation from Center')
ax.set_title('Feature Discrimination Power', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.savefig('/mnt/user-data/outputs/kmeans_comprehensive_analysis.png', dpi=300, bbox_inches='tight')
print("\n✓ Visualization saved!")


# Find most recession-associated cluster
recession_pcts = []
for cluster in range(OPTIMAL_K):
    recession_mask = (df_clean['cluster'] == cluster) & (df_clean['USREC'] == 1)
    recession_pcts.append((recession_mask.sum() / (df_clean['USREC'] == 1).sum()) * 100)

most_recession_cluster = np.argmax(recession_pcts)
least_recession_cluster = np.argmin(recession_pcts)

print(f"\n📊 MOST COMMON DURING RECESSIONS:")
print(f"   Cluster {most_recession_cluster}: {cluster_labels[most_recession_cluster]}")
print(f"   {recession_pcts[most_recession_cluster]:.1f}% of recession months")

print(f"\n📊 LEAST COMMON DURING RECESSIONS:")
print(f"   Cluster {least_recession_cluster}: {cluster_labels[least_recession_cluster]}")
print(f"   {recession_pcts[least_recession_cluster]:.1f}% of recession months")

# Show key features of each
print(f"\n🎵 RECESSION CLUSTER CHARACTERISTICS:")
profile = cluster_profiles.loc[most_recession_cluster]
mean = cluster_profiles.mean()
for feat in ['danceability', 'energy', 'acousticness', 'valence']:
    diff = profile[feat] - mean[feat]
    print(f"   {feat:15s}: {profile[feat]:.3f} ({diff:+.3f} vs mean)")




In [ ]:
"""
K-Means Clustering: Recession vs Normal Comparison
Produces single publication-ready bar chart for poster
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# 1. LOAD DATA & SETUP

df = pd.read_csv('music_econ_topics_merged.csv')

feature_cols = ['danceability', 'energy', 'valence', 'tempo', 
                'acousticness', 'instrumentalness', 'speechiness', 'loudness']

df_clean = df[feature_cols + ['week_date', 'USREC']].dropna()

# 2. RUN K-MEANS CLUSTERING

# Standardize features (required for K-means)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_clean[feature_cols])

# Fit K-means with K=3
kmeans = KMeans(n_clusters=3, random_state=42, n_init=20)
df_clean['cluster'] = kmeans.fit_predict(X_scaled)

# Assign interpretable labels based on cluster analysis
cluster_labels = {
    0: "Balanced",
    1: "High-Energy/Dance",
    2: "Acoustic/Positive"
}

print(f"✓ Clustered {len(df_clean)} months into 3 groups")


fig, ax = plt.subplots(figsize=(10, 6))

# Calculate percentages for each cluster during recession vs normal times
recession_pcts = []
normal_pcts = []
cluster_names = []

for cluster in range(3):
    # Count months in this cluster during recession/normal periods
    recession_in_cluster = ((df_clean['cluster'] == cluster) & (df_clean['USREC'] == 1)).sum()
    normal_in_cluster = ((df_clean['cluster'] == cluster) & (df_clean['USREC'] == 0)).sum()
    
    total_recession = (df_clean['USREC'] == 1).sum()
    total_normal = (df_clean['USREC'] == 0).sum()
    
    recession_pct = (recession_in_cluster / total_recession) * 100
    normal_pct = (normal_in_cluster / total_normal) * 100
    
    recession_pcts.append(recession_pct)
    normal_pcts.append(normal_pct)
    cluster_names.append(cluster_labels[cluster])

# Create grouped bar chart
x = np.arange(len(cluster_names))
width = 0.35

bars1 = ax.bar(x - width/2, recession_pcts, width, label='Recession', 
               color='darkred', alpha=0.8, edgecolor='black', linewidth=1.5)
bars2 = ax.bar(x + width/2, normal_pcts, width, label='Normal', 
               color='steelblue', alpha=0.8, edgecolor='black', linewidth=1.5)

# Add percentage labels on bars
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.1f}%',
                ha='center', va='bottom', fontsize=11, fontweight='bold')

# Formatting
ax.set_xlabel('Cluster Type', fontsize=14, fontweight='bold')
ax.set_ylabel('Percentage of Months (%)', fontsize=14, fontweight='bold')
ax.set_title('K-means Clustering: Music Patterns During Recession vs Normal Times', 
             fontsize=15, fontweight='bold', pad=20)
ax.set_xticks(x)
ax.set_xticklabels(cluster_names, fontsize=12)
ax.legend(fontsize=12, loc='upper left')
ax.grid(True, alpha=0.3, axis='y', linestyle='--')
ax.set_ylim(0, 60)

plt.tight_layout()
plt.savefig('kmeans_recession_comparison.png', dpi=300, bbox_inches='tight')
print("✓ Chart saved as 'kmeans_recession_comparison.png'")
plt.show()



for i, cluster in enumerate(range(3)):
    diff = recession_pcts[i] - normal_pcts[i]
    print(f"\n{cluster_labels[cluster]:20s}:")
    print(f"  Normal:    {normal_pcts[i]:5.1f}%")
    print(f"  Recession: {recession_pcts[i]:5.1f}%")
    print(f"  Change:    {diff:+5.1f} percentage points", end="")
    
    if abs(diff) > 10:
        print(f" {'↑ MAJOR SHIFT' if diff > 0 else '↓ MAJOR SHIFT'}")
    elif abs(diff) > 5:
        print(f" {'↑' if diff > 0 else '↓'}")
    else:
        print(" (minimal change)")

# Statistical test
from scipy.stats import chi2_contingency

contingency_table = pd.crosstab(df_clean['cluster'], df_clean['USREC'])
chi2, p_value, dof, expected = chi2_contingency(contingency_table)

print(f"\nStatistical Test: χ² = {chi2:.2f}, p = {p_value:.4f}")
print(f"Result: {'✓ Significant relationship' if p_value < 0.05 else '✗ Not significant'}")
